In [1]:
# Authors: Lily Picchioni & Andrew Janedy
# March 2025

# Problem Statement: Diabetes in the United States. What living conditions and geographical areas within the United States can lead to an increased risk
# of diabetes?

# Background: Diabetes is a growing conern within The United States, with 37.3 million people currently diagnosed, rates of diagnosed cases of
# diabetes has risen dramatically over the last 20 years. The disease can cause life threatening symptoms and can disrupt an individual's quality of life. 
# Our project has the potential to contribute to the public domain by identifying possible causes of the increase of diabetes in The United States. We
# have combined multiple data sets in an attempt to find potential patterns, including data on diabetes, obesity, poverty and food security in the US.
# Identifying potential relationships between these data sets is the first step in reducing the prevalence of this life threatening disease in the United
# States. If we can draw significant parallels between an individuals ability to obtain quality, healthy foods, and the prevalence of conditions such as 
# obesity and diabetes, this data can be used to assist regions more likely to be impacted by these ailments.

# Relevant data frames:
# - diabetes_data: An entry for each state representing the rate of diabetes for every year between 2000-2022
# --- Index: 'State' (The unique state name and D.C.)
# --- Columns: XXXX (where XXXX represents the numerical year)
# --- Values: 'Total - Percentage' (The % rate of diabetes per capita for the given year)

# Exploratory Data Analysis Documentation:
# 1. Visualizing rates of diabetes, obesity, and poverty per state from 2000 to 2020
    # Plotting raw data is an essential part of exploratory data analysis. We plotted three graphs showing the change in each state's percentage of 
    # diabetes, obesity, and poverty from 2000 to 2020. These graphs will allow us to find patterns and trends within each state. We can also compare the
    # three graphs to find patterns between diabetes, obesity, and poverty rates. 
# 2. Dropping Columns
    # The poverty by state dataset originally had 500+ columns, which was reduced down to only the 33 columns that ask distinct questions
    # and are relevant to our research. This will help us process the data further by clearing out unecessary data and making data analysis more
    # efficient

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
# Processes and combines diabetes rate data from multiple state-specific CSV files.

# This script reads all CSV files in the "Diabetes_Rates" directory. It extracts the 
# state name from the filename, loads the data into a dataframe, adds a "State" column 
# to indicate the data's origin, and finally concatenates all state-specific 
# dataframes into a single dataframe.

diabetes_data_frames = []  # A list to hold the data frame for each state
directory = "Diabetes_Rates"

for filename in os.listdir(directory):  # For all files in Diabetes_Rates directory
    
    if filename.startswith('.'):  # Skip hidden/system files like .ipynb_checkpoints
        continue
        
    file_path = os.path.join(directory, filename)  # Get absolute file path
    state = filename[:-19].replace('_', ' ')# Extract state from filepath
    data_frame = pd.read_csv(file_path)  # Create data frame from file
    data_frame['State'] = state  # Add column to show source state
    diabetes_data_frames.append(data_frame)  # Add state data frame to list of data frames

diabetes_data = pd.concat(diabetes_data_frames)  # Concatenate all the data frames into one

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'Diabetes_Rates'

In [ ]:
print(f"diabetes_data contains {len(diabetes_data)} rows of data")

In [ ]:
diabetes_data.head()

In [ ]:
# Keep only the State, Year, and Total - Percentage columns

columns_to_keep = ['State', 'Year', 'Total - Percentage']
diabetes_data = diabetes_data[columns_to_keep]
diabetes_data.head()

In [ ]:
diabetes_data.info()

In [ ]:
diabetes_data.head()

In [ ]:
# Join all state entries into a single entry with a column for every year

# Pivot the dataframe: each state will have one row, and each year will become a separate column
diabetes_data = diabetes_data.pivot(index='State', columns='Year', values='Total - Percentage')

print(diabetes_data.head)

In [ ]:
# Replace all NaN values with the average of the previous and next available value.
# This method does not account for back to back NaN values

# Iterate over each row (state) in the dataframe
for state in diabetes_data.index:
    # Iterate over each column (year) for that state
    for year in diabetes_data.columns:
        if pd.isna(diabetes_data.loc[state, year]):  # Check if the value is NaN
            prev_value = diabetes_data.loc[state, year - 1] if year - 1 in diabetes_data.columns else None
            next_value = diabetes_data.loc[state, year + 1] if year + 1 in diabetes_data.columns else None
            
            # If both previous and next values exist, replace with their average
            if prev_value is not None and next_value is not None:
                diabetes_data.loc[state, year] = (prev_value + next_value) / 2
            # If only previous value exists, use it
            elif prev_value is not None:
                diabetes_data.loc[state, year] = prev_value
            # If only next value exists, use it
            elif next_value is not None:
                diabetes_data.loc[state, year] = next_value

print(diabetes_data)

In [ ]:
# Replace 'California' with the state you're interested in
state_data = diabetes_data.loc['California']

state_data

In [ ]:
# Set up the plot
plt.figure(figsize=(10, 10))

# Loop through each state and plot its data
for state in diabetes_data.index:
    plt.plot(diabetes_data.columns, diabetes_data.loc[state], label=state)

# Add labels and title
plt.xlabel('Year')
plt.ylabel('Total - Percentage')
plt.title('Diabetes Total - Percentage by State Across Years')

# # Rotate the x-axis labels for better readability
# plt.xticks(rotation=45)

# Add a legend
plt.legend(title="State", bbox_to_anchor=(1.05, 1), loc='upper left')

# Display the plot
plt.tight_layout()  # To ensure everything fits well in the figure
plt.show()

In [ ]:
# Get the first and last year in the dataset
first_year = diabetes_data.columns.min()  # Earliest year
last_year = diabetes_data.columns.max()   # Latest year

# Calculate percent change for each state
# diabetes_data['Percent Change'] = round(((last_year.pivot - first_year.pivot) / first_year.pivot) * 100, 1)

percent_change = round(((last_year - first_year) / first_year) * 100, 1)
print(percent_change)
# print(diabetes_data['Percent Change'])

In [ ]:
# # Sort the dataframe for better visualization
# pivot_df_sorted = diabetes_data.sort_values(by='Percent Change', ascending=True)

# # Plot the percent change
# plt.figure(figsize=(12, 6))
# plt.bar(pivot_df_sorted.index, pivot_df_sorted['Percent Change'])

# # Add labels and title
# plt.xlabel('State')
# plt.ylabel('Percent Change (%)')
# plt.title('Percent Change in Total - Percentage from First to Last Year')
# plt.xticks(rotation=90)  # Rotate state names if needed

# plt.show()

In [ ]:
# Load datasets related to obesity and poverty in the U.S.
# Each dataset contains state-level percentage data for different each category

obesity_data = pd.read_csv("Obesity_Percent_by_State.csv")
poverty_data = pd.read_csv("Poverty_Percent_by_State.csv")

print(f"obesity_data contains {len(obesity_data)} rows of data")
print(f"poverty_data contains {len(poverty_data)} rows of data")

In [ ]:
obesity_data.head()

In [ ]:
obesity_data.set_index("State", inplace=True)

In [ ]:
obesity_data.head()

In [ ]:
# Clean all data of missing values.  Missing values will be replaced by the average
# of the previous and the next value.  If the missing value is the first or last
# entry, it will be replaced with a copy of the next or previous entry, respectively


# Ensure column names are integers
obesity_data.columns = obesity_data.columns.astype(int)

# Ensure column names are integers
obesity_data.columns = obesity_data.columns.astype(int)
# Iterate over each row (state) in the dataframe
for state in obesity_data.index:
    for year in obesity_data.columns:
        if pd.isna(obesity_data.at[state, year]):  # Check if the value is NaN
            prev_year = year - 1
            next_year = year + 1
            
            # Get previous and next values safely
            prev_value = obesity_data.at[state, prev_year] if prev_year in obesity_data.columns else None
            next_value = obesity_data.at[state, next_year] if next_year in obesity_data.columns else None
            
            # Replace NaN with the average of previous and next values
            if prev_value is not None and next_value is not None:
                obesity_data.at[state, year] = (prev_value + next_value) / 2
            elif prev_value is not None:
                obesity_data.at[state, year] = prev_value
            elif next_value is not None:
                obesity_data.at[state, year] = next_value

print(obesity_data)


In [ ]:
# Set up the plot
plt.figure(figsize=(10, 10))

# Loop through each state and plot its data
for state in obesity_data.index:
    plt.plot(obesity_data.columns, obesity_data.loc[state], label=state)

# Add labels and title
plt.xlabel('Year')
plt.ylabel('Total - Percentage')
plt.title('Obesity Total - Percentage by State Across Years')

# # Rotate the x-axis labels for better readability
# plt.xticks(rotation=45)

# Add a legend
plt.legend(title="State", bbox_to_anchor=(1.05, 1), loc='upper left')

# Display the plot
plt.tight_layout()  # To ensure everything fits well in the figure
plt.show()

In [ ]:
poverty_data.head()

In [ ]:
poverty_data.info() # Information about the poverty data set. 

In [ ]:
poverty_data.isnull().sum() # Check how many null values are in the poverty data set

In [ ]:
poverty_data.set_index("State", inplace=True)
poverty_data.head()

In [ ]:
# Ensure we drop the '%Change' column 
plot_data = poverty_data.drop(columns=['%Change'], errors='ignore')  

# Set up the plot
plt.figure(figsize=(12, 8))

# Loop through each state and plot its data
for state in plot_data.index:
    plt.plot(
        plot_data.columns, 
        plot_data.loc[state], 
        label=state, 
        alpha=0.7,  
        linestyle='-' if hash(state) % 2 == 0 else '--'
    )

# Add labels and title
plt.xlabel('Year')
plt.ylabel('Total - Percentage')
plt.title('Poverty Total - Percentage by State Across Years')

# Rotate x-axis labels for better readability
plt.xticks(rotation=45)

# Set y-axis ticks at increments of 5 (ensuring only whole numbers)
y_min, y_max = plt.ylim()  
y_min = int(np.floor(y_min))  
y_max = int(np.ceil(y_max))   
plt.yticks(np.arange(y_min, y_max + 1, 5))  

# Add a legend
plt.legend(title="State", bbox_to_anchor=(1.05, 1), loc='upper left', ncol=2)

# Display the plot
plt.tight_layout()  
plt.show()


In [ ]:
# Load datasets related to food security in the U.S.
# This dataset represents U.S. census data collected in regards to, among other 
# factors, food security as it relates to finances, housing, and employment

_2019_food_security_data = pd.read_csv("Food_Security/Food_Security_2019.csv")
_2020_food_security_data = pd.read_csv("Food_Security/Food_Security_2020.csv")
_2021_food_security_data = pd.read_csv("Food_Security/Food_Security_2021.csv")
_2022_food_security_data = pd.read_csv("Food_Security/Food_Security_2022.csv")
_2023_food_security_data = pd.read_csv("Food_Security/Food_Security_2023.csv")

In [ ]:
_2019_food_security_data.head() # First five rows of the 2019 food security data set

In [ ]:
FoodSecurityList2019 = _2019_food_security_data.columns.tolist() 
FoodSecurityList2020 = _2019_food_security_data.columns.tolist()
FoodSecurityList2021 = _2019_food_security_data.columns.tolist()
FoodSecurityList2022 = _2019_food_security_data.columns.tolist()
FoodSecurityList2023 = _2019_food_security_data.columns.tolist()

In [ ]:
_2019_Food_Security = set(FoodSecurityList2019)  
_2020_Food_Security = set(FoodSecurityList2020)
_2021_Food_Security = set(FoodSecurityList2021)
_2022_Food_Security = set(FoodSecurityList2022)

# Get all unique column names across the datasets
all_columns = _2019_Food_Security | _2020_Food_Security | _2021_Food_Security | _2022_Food_Security  # Union of all sets

# Find common columns (present in all datasets)
common_columns = _2019_Food_Security & _2020_Food_Security & _2021_Food_Security & _2022_Food_Security  # Intersection of all sets

# Find unique columns (columns that appear in at least one dataset but not all)
unique_columns = all_columns - common_columns

print(f"Number of common columns: {len(common_columns)}")
print(f"Total number of columns (set): {len(all_columns)}")
print(f"Columns unique to a single data set: {len(unique_columns)}")

In [ ]:
food_security_dataframes = [] # List to hold food security data sets
food_security_directory = "Food_Security"

for filename in os.listdir(food_security_directory):
    
    if filename.startswith('.') or filename.endswith('.pdf'):  # Skip hidden/system files like .ipynb_checkpoints
        continue
        
    file_path_2 = os.path.join(food_security_directory, filename)
    year = filename[14:-4]
    data_frame_2 = pd.read_csv(file_path_2)
    data_frame_2['Year'] = year
    food_security_dataframes.append(data_frame_2)

food_security_data = pd.concat(food_security_dataframes)

In [ ]:
# NOTICE: The scope of this dataset is described in detail in the technical document
# .pdf files located in the Food_Security sub-directory.  The initial cleaning done 
# on this dataset by keeping a relatively small number of the original 500+ columns
# was determined by analyzing these documents

# A dictionary to contain the column names of the original data sets as keys and the 
# new names to be used for the truncated data frame.  Of the over 500 original columns
# in the data set, only a few dozen will be considered for machine learning

# Standard Key:
#     -2 - Don't Know
#     -3 - Refused
#     -9 - No Response

relevant_columns = {
    'HRHHID': 'ID',  # Survey ID
    'HRYEAR': 'Year',  # Year of record
    'HETENURE': 'Home Owned/Rent/Other',  # Does individual rent, own, or live somewhere free
    'HEHOUSUT': 'Housing Type',  # Type of housing unit, ie house, aparment, dorm, etc
    'HEFAMINC': 'Household Income',  # Total household income
    'HRNUMHOU': 'Household Size',  # Number of household occupants
    'GEREG': 'Region',  # 1 - Northeast, 2 - Midwest, 3 - South, 4 - West
    'GEDIV': 'Division',  # 1 - New England
                          # 2 - Mid Atlantic
                          # 3 - East North Central 
                          # 4 - West North Central 
                          # 5 - South Atlantic
                          # 6 - East South Central
                          # 7 - West South Central
                          # 8 - Mountain
                          # 9 - Pacific
    'GESTFIPS': 'State Abbreviation',  # State abbreviation code
    'PESEX': 'Sex',  # Interviewee sex
    'PEEDUCA': 'Highest Education',  # Interviewee highest education attained1
    'PEMJNUM': '# Jobs Held',  # Interviewee number of jobs held
    'HES8B': 'Food Needs Met',  # Does interviewee need more money to buy food?: 1 - Yes, 2 - No & standard key
    'HESP1': 'SNAP Benefits',  # Has interviewee or a household member recieved snap benefits?: 1 - Yes, 2 - No & standard key
    'HES9': 'Food Security 1',  # Have you run out of money for food in the last 12 months?: 1 - Yes, 2 - No & standard key
    'HESS1': 'Food Security 2',  # Do you often have enough desireable food in the house?: 1 - Always, 2 - Often, 3 - Sometimes, 4 - Rarely & standard key
    'HESS2': 'Food Security 3',  # Do you worry you may run out of money for food?: 1 - Often, 2 - Sometimes, 3 - Never & standard key
    'HESS3': 'Food Security 4',  # Have you run out of food and not had money for more?: 1 - Often, 2 - Sometimes, 3 - Never & standard key
    'HESH2': 'Food Security 5',  # Have you had to reduce meals because of finances?: 1 - Yes, 2 - No & standard key
    'HESH3': 'Food Security 6',  # Have you eaten less because of finances?: 1 - Yes, 2 - No & standard key
    'HESH4': 'Food Security 7',  # Have you gone hungry due to finances?: 1 - Yes, 2 - No & standard key
    'HESH5': 'Food Security 8',  # Have you lost weight due to finances/lack of food?: 1 - Yes, 2 - No & standard key
    'HESS5': 'Food Security 9',  # Purchase low cost food in times of financial hardship (7-26)
    'HESH1': 'Food Security 10',  # Children underfed in times of financial harship (7-27)
    'HESH2': 'Food Security 11',  # Children underfed in times of financial hardship (7-28)
    'HESSH3': 'Food Security 12',  # Children left hungry in times of financial hardship (7-30)
    'HESSH4': 'Food Security 13',  # Children skipping meals in times of financial harship (7-31)
    'HESSH5': 'Food Security 14',  # Children not eating for a whole day (7-33)
    'HESC3': 'Food Security 15',  # Received food from food pantry
    'HESC4': 'Food Security 16',  # Received meals from soup kitchen or shelter
    'HRFS12MD': 'Food Security Status',  # Very Low (4) - High (1) food security
    'HRFS12MC': 'Child Food Security Status',  # Very Low (3) - High (1) food security
}

In [ ]:
# Rename all the columns represented as keys in the relevant_columns dictionary to the
# value with which it is associated
food_security_data.rename(columns=relevant_columns, inplace=True)

# Create a list of the values in relevant columns, these will be the only columns in the 
# cleaned data frame
columns_to_keep = list(relevant_columns.values())

# Remove all of the superfluous columns/data
food_security_data = food_security_data[columns_to_keep]

In [ ]:
print(f"food_security_data column count: {len(food_security_data.columns)}")

In [ ]:
print(len(food_security_data))

In [ ]:
food_security_data.isnull().sum()

In [ ]:
food_security_data = food_security_data.dropna(subset=['State Abbreviation'])